In [1]:
import numpy as np

from utils.data import create_data
from utils.utils import scikit_svm, predict, accuracy, rbf_kernel, Q_matrix, is_pos_def
from svm import SVM

import matplotlib.pyplot as plt

import time

In [2]:
%matplotlib widget

In [35]:
#create_data(file_name = 'data_38', classes = [3,8])
load_data = np.load('data_38.npz')
train_X, train_y = load_data['train_X'], load_data['train_y']
test_X, test_y = load_data['test_X'], load_data['test_y']

# Checking if data is linearly separable

In [52]:
# Takes a bit to run
#linear = scikit_svm('linear', train_X, train_y, test_X, test_y)
rbf = scikit_svm('rbf', train_X, train_y, test_X, test_y)
#print(f'Accuracy using linear kernel: {linear}')
print(f'Accuracy using gaussian kernel: {rbf}')

Accuracy using gaussian kernel: 0.968


In [38]:
# Create mini dataset for testing
train_X, train_y = train_X[:1000,:], train_y[:1000]
test_X, test_y = test_X[:500,:], test_y[:500]

# Running SMO

## Get accuracy before training

In [4]:
num_training_examples = train_y.shape[0]

alpha = np.ones(num_training_examples)
b = 0
kernel_matrix = rbf_kernel(train_X, test_X)

pred_y = predict(alpha, kernel_matrix, b, train_y)

print(f'Accuracy before training: {accuracy(pred_y, test_y)}')

Accuracy before training: 0.5090725806451613


# Speed test: SMO fast and slow

In [51]:
start = time.time()
svm = SVM(optim = 'SMO-fast',
          kernel = 'rbf',
          C = 1,
          max_passes = 10,
          calc_g_iterates = False)
svm.fit(train_X, train_y)
print(svm.get_accuracy(test_X, test_y))
end = time.time()
print(f'Time taken with fast implementation: {end-start}s')

0.93
Time taken with fast implementation: 1.151731014251709s


In [46]:
start = time.time()
svm = SVM(optim = 'SMO-slow',
          kernel = 'rbf',
          C = 1,
          max_passes = 6,
          calc_g_iterates = False)
svm.fit(train_X, train_y)
print(svm.get_accuracy(test_X, test_y))
end = time.time()
print(f'Time taken with slow implementation: {end-start}s')

0.934
Time taken with slow implementation: 0.5690391063690186s


# Check if Q is positive definite
$Q$ is the matrix with elements $Q_{ij} = y_iy_jk(x_i,x_j)$

In [18]:
Q_mat = Q_matrix(svm.train_X, svm.train_y, rbf_kernel)

In [19]:
is_pos_def(Q_mat)

True

# Checking the convergence of g

\begin{equation}
  g(\alpha) = \sum_{i=1}^m \alpha_i - \frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m \alpha_i \alpha_j y_i y_j k(x_i,x_j)
   \label{eq:svoptim_problem}
\end{equation}

In [22]:
svm.plot_convergence()
plt.xlabel('Steps')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Steps')